# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    spread_cls = []
    spread_avg = []
    
    tick_opn = []
    tick_hig = []
    tick_low = []
    tick_cls = []
    tick_avg = []
    tick_sd = []
    tick_ema_10 = []
    tick_ema_25 = []
    tick_ema_50 = []
    tick_ema_75 = []
    tick_ema = []
    tick_slope = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])      
        
        spread_cls.append(spread_list[-1])
        spread_avg.append(np.mean(spread_list))

        tick_opn.append(tick_list[0])
        tick_hig.append(np.max(tick_list))
        tick_low.append(np.min(tick_list))
        tick_cls.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))  
        tick_sd.append(np.std(tick_list))
        
        l = tick_list[-(int(len(tick_list)*0.10)):]
        tick_ema_10.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.25)):]
        tick_ema_25.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.50)):]
        tick_ema_50.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.75)):]
        tick_ema_75.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        tick_ema.append(list(pd.DataFrame(tick_list).ewm(span=len(tick_list)).mean()[0])[len(tick_list) - 1]) 
        tick_slope.append(get_slope(tick_list))
        
    temp_df['spread_avg'] = spread_avg  
    temp_df['spread_cls'] = spread_cls  

    temp_df['tick_opn'] = tick_opn
    temp_df['tick_high'] = tick_hig
    temp_df['tick_low'] = tick_low
    temp_df['tick_cls'] = tick_cls
    temp_df['tick_avg'] = tick_avg  
    temp_df['tick_sd'] = tick_sd
    
    temp_df['tick_ema_10'] = tick_ema_10  
    temp_df['tick_ema_25'] = tick_ema_25  
    temp_df['tick_ema_50'] = tick_ema_50  
    temp_df['tick_ema_75'] = tick_ema_75  
    temp_df['tick_ema'] = tick_ema 
    temp_df['tick_slope'] = tick_slope 

    return(temp_df)

In [3]:
def get_slope(y_axis):
    global data
    x_axis = []
    for i in range(len(y_axis)):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))

    return(slope_tick)

## File paths

In [4]:
year = 2019
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [5]:
data = {}
diff_col = 'sema'
data['pip_diff'] = 0.00002

data['number_of_ticks'] = 100

data['sma_len'] = 20
data['lma_len'] = 50

data['rsi_window'] = 14

In [6]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 792 ms


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


In [7]:
%%time
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')
df = pd.read_csv(chunk_file_path)
df.head()

100%|█████████████████| 10000/10000 [00:52<00:00, 190.35it/s]


Records : 10000
Wall time: 52.9 s


,spread_avg,spread_cls,tick_opn,tick_high,tick_low,tick_cls,tick_avg,tick_sd,tick_ema_10,tick_ema_25,tick_ema_50,tick_ema_75,tick_ema,tick_slope
0,0.000447,0.00029,1.146400,1.146740,1.146220,1.146675,1.146540,0.000155,1.146674,1.146682,1.146669,1.146630,1.146599,20.112509
1,0.000469,0.00063,1.146680,1.146770,1.146380,1.146395,1.146541,0.000138,1.146392,1.146396,1.146402,1.146442,1.146476,-22.865939
2,0.000526,0.00038,1.146375,1.146425,1.146215,1.146260,1.146338,0.000080,1.146260,1.146284,1.146311,1.146311,1.146322,-6.257425
3,0.000363,0.00031,1.146255,1.146390,1.146215,1.146225,1.146263,0.000031,1.146221,1.146237,1.146249,1.146256,1.146259,-0.924938
4,0.000618,0.00032,1.146220,1.146410,1.146210,1.146220,1.146375,0.000047,1.146279,1.146330,1.146357,1.146367,1.146370,-0.461210


In [8]:
%%time

df['diff_next_high'] = df['tick_high'].shift(-1) - df['tick_cls']
df['diff_next_low'] = df['tick_low'].shift(-1) - df['tick_cls']

df['dir'] = 'same'
df.loc[abs(df['diff_next_high']) > abs(df['diff_next_low']), 'dir'] = 'high'
df.loc[abs(df['diff_next_low']) > abs(df['diff_next_high']), 'dir'] = 'low'

df['direction'] = 'same'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] <= -0.0002), 'direction'] = 'decrease'

df.loc[(df['dir'] == 'low') & (df['diff_next_low'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'low') & (df['diff_next_low'] <= -0.0002), 'direction'] = 'decrease'

Wall time: 17 ms


In [9]:
%%time
col_list = ['tick_opn', 'tick_high', 'tick_low', 'tick_cls', 'tick_avg', 'tick_ema', 'spread_cls', 'tick_ema_10', 'tick_ema_25', 'tick_ema_50', 'tick_ema_75']
for i, val in enumerate(col_list):    
    df[val+'_diff'] =  df[val].diff()
    del df[val]    
    
df['ema_gap'] = df['tick_ema_10_diff'] - df['tick_ema_diff']    

Wall time: 15 ms


In [10]:
%%time
data['rs_max'] = 1e6
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['tick_avg_diff'] > 0] = abs(df['tick_avg_diff'])
df['loss'].loc[df['tick_avg_diff'] < 0] = abs(df['tick_avg_diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

Wall time: 16 ms


In [11]:
col_order = [
 'spread_avg',
 'spread_cls_diff', 
 'tick_opn_diff',
 'tick_high_diff',
 'tick_low_diff',
 'tick_cls_diff',
 'tick_avg_diff',
 'tick_sd',
 'tick_ema_10_diff',
 'tick_ema_25_diff',
 'tick_ema_50_diff',
 'tick_ema_75_diff',
 'tick_ema_diff',
 'ema_gap',
 'gain',
 'loss',
 'avg_gain',
 'avg_loss',
 'rsi',
 'tick_slope',
 'direction'
 ]

df = df[col_order]
df = df.dropna()

## Write data to csv

In [12]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 367 ms


In [13]:
print(f'Total records : {len(df)}')
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head()

Total records : 9987
          counts  percentage
same        7887   78.972664
decrease    1071   10.723941
increase    1029   10.303394


,spread_avg,spread_cls_diff,tick_opn_diff,tick_high_diff,tick_low_diff,tick_cls_diff,tick_avg_diff,tick_sd,tick_ema_10_diff,tick_ema_25_diff,...,tick_ema_75_diff,tick_ema_diff,ema_gap,gain,loss,avg_gain,avg_loss,rsi,tick_slope,direction
13,0.000405,0.00025,0.000025,0.000130,-0.000020,0.000055,0.000035,0.000060,0.000056,0.000071,...,0.000063,0.000053,0.000003,0.000035,0.000000,0.000019,0.000033,36.630037,8.526058,same
14,0.000531,-0.00029,0.000050,0.000065,0.000030,0.000145,0.000057,0.000058,0.000099,0.000021,...,0.000010,0.000027,0.000072,0.000057,0.000000,0.000023,0.000033,41.189802,-3.724729,decrease
15,0.000375,0.00012,0.000145,-0.000010,-0.000275,-0.000430,-0.000132,0.000125,-0.000373,-0.000296,...,-0.000204,-0.000179,-0.000194,0.000000,0.000132,0.000023,0.000043,35.201212,-19.796746,same
16,0.000522,0.00025,-0.000400,-0.000305,-0.000005,-0.000045,-0.000201,0.000065,-0.000097,-0.000086,...,-0.000144,-0.000164,0.000066,0.000000,0.000201,0.000023,0.000043,35.277537,-6.967128,same
17,0.000633,0.00014,-0.000055,-0.000015,-0.000010,0.000070,0.000020,0.000056,0.000092,0.000068,...,0.000061,0.000045,0.000047,0.000020,0.000000,0.000025,0.000037,39.812039,3.870720,increase
